In [1]:
import pandas as pd
import numpy as np


In [2]:
df_assessments = pd.read_csv('E:/Soft/jupyter/project/assessments(1).csv')
df_courses = pd.read_csv('E:/Soft/jupyter/project/courses(1).csv')
df_student_assessments = pd.read_csv('E:/Soft/jupyter/project/studentAssessment(1).csv')
df_student_registration = pd.read_csv('E:/Soft/jupyter/project/studentRegistration(1).csv')

In [1]:
#перед началом проекта все датасеты были просмотренны визуально и проверены на пропуски.

Сколько студентов успешно сдали только один курс? (Успешная сдача — это зачёт по курсу на экзамене)

In [4]:
# для ответа на этот вопрос нам помогут два датафрейма df_student_assessments и df_assessments
# так как в них есть все необходимые данные

In [38]:
# объединим два датафрейма в один по ключу 'id_assessment'. из df_assessments нам нужен только тип экзамена. 
df_exams_with_scores = pd.merge(df_student_assessments, 
                                df_assessments[df_assessments.assessment_type == 'Exam'], 
                                on = 'id_assessment')

In [6]:
# получаем датафрейм следующего вида.
df_exams_with_scores.head()

,id_assessment,id_student,date_submitted,is_banked,score,code_module,code_presentation,assessment_type,date,weight
0,24290,558914,230,0,32.0,CCC,2014B,Exam,NaN,100.0
1,24290,559706,234,0,78.0,CCC,2014B,Exam,NaN,100.0
2,24290,559770,230,0,54.0,CCC,2014B,Exam,NaN,100.0
3,24290,560114,230,0,64.0,CCC,2014B,Exam,NaN,100.0
4,24290,560311,234,0,100.0,CCC,2014B,Exam,NaN,100.0


In [7]:
# осталось ответить на поставленный вопрос. сначала сделаем запрос, который отсечет неудачные экзамены (то есть <40)
# сгруппируем по id_student, посчитаем количество сданных экзаменов по столбцу id_assessment,
# сделаем запрос, отвечающий на поставленный вопрос (сколько студентов сдали только ОДИН курс),
# и для красоты дадим колонке подходящее имя.
df_exams_with_scores.query('score >= 40.0').groupby('id_student', as_index = False)\
                    .agg({'id_assessment':'count'})\
                    .query('id_assessment == 1')\
                    .rename(columns = {'id_assessment':'success_exams'})
# на выходе получаем перечень id_student, которые успешно сдали 1 экзамен.

,id_student,success_exams
0,23698,1
1,24213,1
2,27116,1
3,28046,1
4,28787,1
...,...,...
4092,2694886,1
4093,2694933,1
4094,2695608,1
4095,2697181,1


In [8]:
# ответ: только 3802 студента сдали успешно только 1 курс.

Выяви самый сложный и самый простой экзамен: 
найди курсы и экзамены в рамках курса, которые обладают самой низкой и самой высокой завершаемостью*

*завершаемость = кол-во успешных экзаменов / кол-во всех попыток сдать экзамен

In [9]:
# для ответа на этот вопрос у нас уже есть хороший датафрейм, который мы объединили в прошлом шаге - df_exams_with_scores

In [41]:
# создадим новый датафрейм из запросов к df_exams_with_scores

# мержить будем к датафрейму, в который мы отберем из df_exams_with_scores количество успешно сданных экзаменов.

# сначала через запрос отберем сданные экзамены (score >=40),
# затем сгруппируем сначала по code_module, затем по id_assessment,
# проведем аггрегацию по id_student, подсчитав таким образом сколько раз был сдан тот или иной экзамен по тому или иному предмету.
# и для удобства сразу переименуем колонку id_student, чтобы при объединении небыло одинаковых колонок.

# а примерживать будем запрос из df_exams_with_scores, в который мы отберем просто все попытки сдать экзамен.
# логика как и выше, только без запроса query. только группировка, аггерация и переименование.

# мержить будем по ключам code_module и id_assessment

In [11]:
df_total_exams = pd.merge(df_exams_with_scores.query('score >= 40')\
                                              .groupby(['code_module', 'id_assessment'], as_index = False)\
                                              .agg({'id_student':'count'})\
                                              .rename(columns = {'id_student':'success'}),
                          df_exams_with_scores.groupby(['code_module', 'id_assessment'], as_index = False)\
                                              .agg({'id_student':'count'})\
                                              .rename(columns = {'id_student':'all'}),
                          on = ['code_module', 'id_assessment'])

In [12]:
# проверим что получилось
df_total_exams

,code_module,id_assessment,success,all
0,CCC,24290,664,747
1,CCC,24299,1019,1168
2,DDD,25340,504,602
3,DDD,25354,878,968
4,DDD,25361,485,524
5,DDD,25368,842,950


In [13]:
# отлично. теперь узнаем долю успешно сданных экзаменов относительно всех попыток сдачи экзаменов
# создадим колонку с соответствующим показателем и отфильтруем по убыванию.
df_total_exams['completeness'] = df_total_exams['success'] / df_total_exams['all']
df_total_exams.sort_values('completeness', ascending = False)

,code_module,id_assessment,success,all,completeness
4,DDD,25361,485,524,0.925573
3,DDD,25354,878,968,0.907025
0,CCC,24290,664,747,0.888889
5,DDD,25368,842,950,0.886316
1,CCC,24299,1019,1168,0.872432
2,DDD,25340,504,602,0.837209


In [14]:
# ответ: по курсу CCC самый сложный экзамен - 24299, самый простой - 24290
# по курсу DDD самый сложный - 25340, самый простой - 25361
# если бы список был огромный, можно было бы узнать через запросы query и выполнив метод min/max
# но так как в таблице всего 6 строк - можно и глазами увидеть. 

По каждому предмету определи средний срок сдачи экзаменов (под сдачей понимаем последнее успешное прохождение экзамена студентом)

In [15]:
# для ответа на этот вопрос вновь переиспользуем датафрейм df_exams_with_scores созданный в первом шаге.
# в нем есть все нужные нам данные, включая "дата сдачи теста студентом, измеряемая как количество дней с начала семестра".

In [16]:
# сначала сделаем запрос, отсеяв успешно сданные экзамены.
# затем сгруппируем по коду предмета
# и сделаем аггрегацию date_submitted, найдя среднее.
df_exams_with_scores.query('score >= 40')\
                    .groupby('code_module', as_index = False)\
                    .agg({'date_submitted':'mean'})
# на выходе получаем среднее количество дней с начала курса и до его успешного завершения по каждому предмету.

,code_module,date_submitted
0,CCC,239.345217
1,DDD,237.976006


In [42]:
# ответ: по предмету CCC среднее количество дней 239,
# по предмету DDD среднее количество дней 238

Выяви самые популярные курсы (ТОП-3) по количеству регистраций на них. А также курсы с самым большим оттоком (ТОП-3).

In [18]:
# для ответа на этот вопрос используем датафрейм df_student_registration, в нем есть нужная нам информация.

In [19]:
# сначала сгруппируем по code_module
# затем сделаем аггрегацию, подсчитав сколько всего было регистраций на каждый code_module
# для удобства переименуем соответствующе переименуем колонку
# отсортируем результат по total_registrations
# и выведем первые 3 результата.
df_student_registration.groupby('code_module', as_index = False)\
                       .agg({'date_registration':'count'})\
                       .rename(columns = {'date_registration':'total_registrations'})\
                       .sort_values('total_registrations', ascending = False)\
                       .head(3)

,code_module,total_registrations
1,BBB,7900
5,FFF,7751
3,DDD,6257


In [20]:
# ответ: топ 3 популярных курсов занимают курсы BBB, FFF, DDD

In [21]:
# для ответа на вторую часть вопроса делаем почти всё то же самое, только работаем уже с колонкой date_unregistration

df_student_registration.groupby('code_module', as_index = False)\
                       .agg({'date_unregistration':'count'})\
                       .rename(columns = {'date_unregistration':'total_unregistrations'})\
                       .sort_values('total_unregistrations', ascending = False)\
                       .head(3)

,code_module,total_unregistrations
5,FFF,2380
1,BBB,2377
3,DDD,2235


In [22]:
# ответ: топ 3 курсов по оттоку занимают курсы FFF, BBB, DDD. 
# собственно самые популярные курсы и имеют самый большой отток.

Напиши функцию на python, позволяющую строить когортный (семестровый) анализ. В период с начала 2013 по конец 2014 выяви семестр с самой низкой завершаемостью курсов и самыми долгими средними сроками сдачи курсов.

In [23]:
# когортами у нас будут выступать семестры.

In [43]:
# объявим функцию, которая на вход будет принимать 1 уже объединенный датафрейм 
# df_student_assessments + df_assessments по ключу id_assessment. в этих датафреймах есть вся нужная нам информация.
# конечно можно объявить функцию, которая будет принимать на вход эти два датафрейма, и уже внутри их объединять:
# def some_function(df1, df2):
#     merged_df = pd.merge(df1, df2, on = 'id_assessment')
# а дальше уже всё как в функции ниже.
# но представим что мы хотим скормить функции уже очищенный и обработанный датафрейм,
# и она даст ответы на интересующие нас вопросы.

# df_student_assessments + df_assessments почему именно эти датафреймы:
# в df_student_assessments записана какая либо активность студентов во время учебы,
# то есть в отличии от df_student_registration где студенты могли отменить регистрацию даже и не начав учиться,
# здесь мы можем подсчитать уникальных студентов, которые поступали на курсы и пытались учиться,
# и в следствии узнать какая доля студентов из общего числа поступивших в итоге закончила курсы.
# c df_assessments всё просто, тут нужные нам данные о тестах.

# ВАЖНАЯ ИНФОРМАЦИЯ НИЖЕ ОТВЕТА.

In [44]:
def cohort_analysis(df):
    
    #создадим датафрейм из запросов, для последующего объединения.
    
    #сгруппируем сначала по когортам, затем по предметам
    #подсчитаем количество уникальных студентов всего
    #для удобства переименуем колонку
    #затем сделаем запрос, который отсеет успешно законченные курсы
    #так же сгруппируем
    #так же подсчитаем
    #и переименуем 
    #объединяем по ключам
 
    df_share_of_succees = pd.merge(df.groupby(['code_presentation', 'code_module'], as_index = False)\
                                     .agg({'id_student':'nunique'})\
                                     .rename(columns = {'id_student':'total_students'}),
                                   df.query('assessment_type == "Exam" and score >= 40')\
                                     .groupby(['code_presentation', 'code_module'], as_index = False)\
                                     .agg({'id_student':'nunique'})\
                                     .rename(columns = {'id_student':'success_students'}),
                                   on = ['code_presentation', 'code_module'])
    
    #теперь к ранее созданному датафрейму присоединяем еще один, со средним количеством дней сдачи курсов.
    
    #запросом отсеиваем успешно сданные курсы
    #группируем по когортам, затем по предметам
    #подсчитаем среднее количество дней на сдачу курса
    #переименуем для удобства
    #объединяем по этим ключам
    
    df_success_and_avg_date = pd.merge(df_share_of_succees,
                                       df.query('assessment_type == "Exam" and score >= 40')\
                                         .groupby(['code_presentation', 'code_module'], as_index = False)\
                                         .agg({'date_submitted':'mean'})\
                                         .rename(columns = {'date_submitted':'avg_date'}),
                                       on = ['code_presentation', 'code_module'])
    
    #теперь в этом общем датафрейме добавим колонку, которая отображает долю завершаемости курса
    #количество студентов успешно сдавших курс деленное на общее количество студентов.
    
    df_success_and_avg_date['share_of_succees'] = df_success_and_avg_date.success_students / df_success_and_avg_date.total_students
    
    #и сделаем так, чтобы функция выводила нужную нам информацию.
    
    print('Семестр с самой низкой завершаемостью курсов: ' + \
          df_success_and_avg_date.sort_values('share_of_succees').iloc[1]['code_presentation'] + \
         ' (' + str(df_success_and_avg_date.sort_values('share_of_succees').iloc[1]['share_of_succees']) + ')')
    print('Семестр с самыми долгими средними сроками сдачи курсов: ' + \
          df_success_and_avg_date.sort_values('avg_date').iloc[1]['code_presentation'] + \
         ' (' + str(df_success_and_avg_date.sort_values('avg_date').iloc[1]['avg_date']) + ')')       

In [45]:
dftest = pd.merge(df_student_assessments, df_assessments, on = 'id_assessment')

In [46]:
cohort_analysis(dftest)

Семестр с самой низкой завершаемостью курсов: 2013B (0.4732394366197183)
Семестр с самыми долгими средними сроками сдачи курсов: 2014B (231.58132530120483)


In [28]:
# в датафрейме df_student_registration есть немаловажный столбец - date_unregistration.
# если в ней пропущено значение - значит студент закончил курс (то есть успешно сдал экзамен).
# если объединить df_student_assessments, df_assessments (по id_assessment) и df_student_registration (по id_students)
# то можно обнаружить некоторое различие в показаниях. то есть получается если мы сделаем запрос из такого датафрейма 
# только строчки где date_unregistration отсутствует - то получаем 4632 уникальных студентов, которые закончили курс,
# НО если в запрос добавить дополнительный фильтр - score >= 40 (так как завершение курса - это успешная сдача экзамена,
# а успешная сдача экзамена - это 40 и более баллов за него), то получается 4097, то есть столько сколько и в функции.
# отсюда вывод - что в датафрейме есть студенты, которые не сдали экзамен, но каким то образом закончили курс.
# тут где то ошибка. только где - не понимаю.

# pd.merge(df_student_assessments, df_assessments[df_assessments.assessment_type == 'Exam'], on = 'id_assessment')\
# .merge(df_student_registration, on = 'id_student')\
# .fillna(0).query('date_unregistration == 0')\
# .nunique()

# pd.merge(df_student_assessments, df_assessments[df_assessments.assessment_type == 'Exam'], on = 'id_assessment')\
# .merge(df_student_registration, on = 'id_student')\
# .fillna(0).query('date_unregistration == 0 and score >= 40')\
# .nunique()

Часто для качественного анализа аудитории использую подходы, основанные на сегментации. Используя python, построй адаптированные RFM-кластеры студентов, чтобы качественно оценить свою аудиторию. В адаптированной кластеризации можешь выбрать следующие метрики: R - среднее время сдачи одного экзамена, F - завершаемость курсов, M - среднее количество баллов, получаемое за экзамен. Подробно опиши, как ты создавал кластеры. Для каждого RFM-сегмента построй границы метрик recency, frequency и monetary для интерпретации этих кластеров. Пример такого описания: RFM-сегмент 132 (recency=1, frequency=3, monetary=2) имеет границы метрик recency от 130 до 500 дней, frequency от 0.1 до 0.5, monetary от 55 до 72 баллов.

In [29]:
# сначала подготовим данные:
# возьмем данные из датафрейма, созданного еще вначале проекта,
# сначала сгруппируем по студентам (потому что проводим анализ аудитории)
# затем подсчитаем среднее количество дней сдачи экзамена и баллов за экзамены.
# для удобства переименуем колонку

# затем сделаем запрос успешно завершенных курсов (завершенный курс = успешная сдача экзамена = количество баллов за экзамен > 40)
# сгруппируем по студентам
# переименуем колонку для удобства
# будем присоединять к "левой" таблице, чтобы не потерять из правой пустые значения с количеством законченных курсов
# и заполним пропущенные значения нулями, ведь если 0 - не закончил ни один курс.

In [30]:
df_rfm = pd.merge(df_exams_with_scores.groupby('id_student', as_index = False)\
                                      .agg({'date_submitted':'mean', 'score':'mean'})\
                                      .rename(columns = {'date_submitted':'avg_date', 'score':'avg_score'}),
                  df_exams_with_scores.query('score >= 40')\
                                      .groupby('id_student', as_index = False)\
                                      .agg({'date_submitted':'count'})\
                                      .rename(columns = {'date_submitted':'completed_courses'}),
                  how = 'left', on = 'id_student').fillna(0)

In [31]:
df_rfm

,id_student,avg_date,avg_score,completed_courses
0,23698,243.0,80.0,1.0
1,24213,236.0,58.0,1.0
2,27116,243.0,96.0,1.0
3,28046,237.0,40.0,1.0
4,28787,243.0,44.0,1.0
...,...,...,...,...
4628,2694886,236.0,69.0,1.0
4629,2694933,230.0,73.0,1.0
4630,2695608,237.0,73.0,1.0
4631,2697181,230.0,80.0,1.0


In [32]:
# создадим пустые колонки для RFM
df_rfm['R'] = ''
df_rfm['F'] = ''
df_rfm['M'] = ''

In [33]:
# и раздадим кластеры для метрик:

# R - среднее время сдачи одного экзамена.
# в датафрейме df_assessment для экзаменов есть время сдачи, то есть крайний срок сдачи. но во первых - не для всех экзаменов
# а во вторых - для каждого экзамена свой дедлайн. для этого пришлось бы обработать датафрейм таким образом, чтобы
# в RFM были не просто студенты, а студенты с пометкой курсов, и у каждого был бы свой определенный показатель R, 1-2-3.
# то есть в зависимости от курса - одно и то же среднее количество дней давало бы разные уровни метрики.
# поэтому здесь кластеры я поделил таким образом:
# 1 - те кто сдают раньше срока. те студенты кто преуспевают в учебе и сдают экзамен до дедлайна.
# 2 - те кто сдают вовремя (или почти вовремя)
# 3 - те кто сдают с опозданием.

df_rfm.loc[df_rfm.avg_date < 240, 'R'] = 1
df_rfm.loc[df_rfm.avg_date > 261, 'R'] = 3
df_rfm.R = df_rfm.R.replace('', 2)

# F - завершаемость курсов.
# здесь всё просто - есть студенты которые закончил курс, есть те кто закончили даже 2 курса, а есть те кто не закончил вообще.
# 1 - те кто закончил больше 1 курса
# 2 - те кто закончил 1 курс
# 3 - те кто не закончил ни одного.

df_rfm.loc[df_rfm.completed_courses > 1, 'F'] = 1
df_rfm.loc[df_rfm.completed_courses < 1, 'F'] = 3
df_rfm.F = df_rfm.F.replace('', 2)

# M - среднее количество баллов за экзамен.
# 1 - те кто набрали больше 80 баллов (то есть близко к "отлично")
# 2 - те кто сдал экзамен в целом
# 3 - те кто набрал меньше 40 баллов (не сдал)

df_rfm.loc[df_rfm.avg_score < 40, 'M'] = 3
df_rfm.loc[df_rfm.avg_score > 80, 'M'] = 1
df_rfm.M = df_rfm.M.replace('', 2)

In [34]:
df_rfm

,id_student,avg_date,avg_score,completed_courses,R,F,M
0,23698,243.0,80.0,1.0,2,2,2
1,24213,236.0,58.0,1.0,1,2,2
2,27116,243.0,96.0,1.0,2,2,1
3,28046,237.0,40.0,1.0,1,2,2
4,28787,243.0,44.0,1.0,2,2,2
...,...,...,...,...,...,...,...
4628,2694886,236.0,69.0,1.0,1,2,2
4629,2694933,230.0,73.0,1.0,1,2,2
4630,2695608,237.0,73.0,1.0,1,2,2
4631,2697181,230.0,80.0,1.0,1,2,2


In [35]:
# мы разбили студентов на RFM кластеры.

In [36]:
# теперь если мы сделаем запрос следующего вида:
df_rfm.query('R == 1 and F == 1 and M == 1')
# то сможем выявить особо преуспевающих в учебе студентов,
# которые сдали 2 курса, экзамены сдавали раньше или не позже дедлайнов, и имели в среднем оценку близкую к "отлично".
# таким образом мы можем, например, отобрать этих студентов, и предложить им еще курсы, подходящие им (если мы продаем курсы),
# или предложить трудоустройство (если мы учебное заведение)

# или же наоборот обратить внимание на 3-3-3 студентов, и найти причину их плохой успеваемости,
# возможно плохая подача материала, или недоступность инструментов и тд.

,id_student,avg_date,avg_score,completed_courses,R,F,M
166,145640,239.5,93.5,2.0,1,1,1
210,172965,236.0,89.0,2.0,1,1,1
634,383347,230.0,83.5,2.0,1,1,1
653,387963,239.0,90.5,2.0,1,1,1
892,445917,233.5,98.0,2.0,1,1,1
933,467230,232.0,84.0,2.0,1,1,1
1027,480929,230.5,85.0,2.0,1,1,1
1172,502554,229.5,96.0,2.0,1,1,1
1328,522312,233.5,86.0,2.0,1,1,1
1425,532718,232.0,84.0,2.0,1,1,1
